# R2lab's Protocol Evaluation (B.A.T.M.A.N. *vs* OLSR)

NOTE: One datasample has been collected and is available in this very directory in `datasample`.

## Purpose

R2lab's Protocol Evaluation is an experience that has been designed to satisfy several purposes.

* Its primary scientific purpose is to compare two wireless mesh routing protocol : Batman and Olsr.
* Second, it serves as an exemple of what we can do with R2lab and all its tools, like nepi-ng. 
* Its third purpose is to illustrate how similar results can be presented inside a Jupyter Notebook.

To do so, the experiment has been split into two parts:
* Data acquisition; this has been automated inside a customizable script. You can run it with a number of different arguments - that typically describe the environment that you want to use.
* Visualization; the second part of this notebook shows how the data gathered can be interactivelyt visualized.

For your convenience, this git repo contains pre-gathered data, that you can visualize right away, without the need to go through the full process of account creation, reservation and so on.

Thanks to the fact that R2lab is an anechoic chamber, and to the information you can find in this notebook, if the hardware keeps the same properties, the experiment should be 100% reproducible.

## Experiment  description

The main core of the experimentation is a simple ping from the chosen experimentation nodes to every other enabled nodes. Yet you can enable or disable multiple other modules : 

1. A tcpdump capture to do more in-depth analysis of your results. As a side note, it also parses it in order to create a mini textual heat map of the chamber.
1. Add the generation of intra channel interference by generating a white noise on the channel with a defined gain.
1. Generate a snapshot of the routing tables right before launching the ping.
1. Launch pseudo-services that will sample the routing tables on the nodes every X ms.
1. Add a first non result driven round of ping before the experiment in order to avoid pollution from previous experiment.

Thanks to the pings, we can have information about the Packet Delivery Ratio (PDR) and the Round-Trip delay Time (RTT).

## Generic properties about Hard/Software on R2lab

### R2lab's Room

The experiment is designed to be launched in the r2Lab's Anechoic chamber. In the following picture, you can see its physical topology.
![r2Lab topology](https://r2lab.inria.fr/assets/img/status-chamber.png)

In this experiment we will use the positions of the different nodes and the obstacle that are present.

### Nodes

They are based on NitosX50 and we are interested by :
* Its state of the art motherboard

   * CPU Intel Core i7-2600 processor
   * 8Gb RAM
   * 240 Gb SSD

* Wireless Interface, dedicated to experimentation, 3 antennas :

    * one Atheros 802.11 93xx a/b/g/n - exposed as atheros


### USRP Nodes (signal noise generators)

In this experiment we use USRP n210 nodes and USRP 2.
Both these nodes emit at a power of 15 dBm and their specification can be found here: https://www.ettus.com/content/files/07495_Ettus_N200-210_DS_Flyer_HR_1.pdf

## Specifics to this experiment

### OLSR implementation

The script will install the latest version of the olsrd service. At the time where the data were collected, the version was : **0.6.6.2-1ubuntu1**

### Batman implementation

The script will install the latest version of the batmand service. At the time where the data were collected, the version was : **0.3.2-17**

### OS'es and images

The experiment relies on 2 different images:

* regular nodes - the ones that participate only with their WiFi interface - run an image named **`batman-olsr`**  
  this image is based on `Ubuntu 16.04.4 LTS` and comes with kernel:  
  <span style="font-size:small">`Linux fit01 4.4.0-21-generic #37-Ubuntu SMP Mon Apr 18 18:33:37 UTC 2016 x86_64`</span>

* the scrambler node, when interferences are enabled, run an image named **`batman-olsr-gnuradio`**  
  this image is also based on `Ubuntu 16.04.4 LTS`, it embeds gnuradio version `3.7.11.1`` and comes with kernel:  
  <span style="font-size:small">`Linux fit05 4.8.0-59-lowlatency #64-Ubuntu SMP PREEMPT Thu Jun 29 21:04:20 UTC 2017 x86_64`</span>

### Discussion on other settings

Our main challenge will be to find experimental settings that allow to trigger multi-hop routing; this is not obvious because of the relatively small size of the room. 

Here's a discussion on the various experimental settings that can be considered, and how we have picked them:

* **set of nodes** first, the set of nodes that participate in the ad hoc network is of course key; as we will see, we have selected one subset of nodes that, taking advantage of the room peculiarities like pillars, distances and other line-of-sight relationships, is a fair background for the experiment. Essentially this is a constant in our study; using the current notebook, researchers should be, for example, able to investigate whether other subsets could be suitable as well;

* **protocol** of course, selecting the protocol among OLSR and batman is an obvious choice when collecting data;

* **interference** - the primary influential setting that allows us to create diverse routing patterns, is our ability to emit noise on the experimental wireless channel; the level of that interference noise is the key external parameter that allows to actually create multi-hop routing schemes; without that noise, all node pairs essentially can talk directly to one another. Note that for that purpose, we selected one node that has a USRP board - namely node 30 ; again this is a parameter in the code, and external researchers have the ability to check if the location of the scrambler node - the one that emits noise - has an impact or not on the results, but as far as we are concerned, we always use the same node for that purpose.

* **wireless settings** - the code also comes with all the usual wireless settings - channel, tx_power, number of antennas, etc.. - as parameters, but as far as we are concerned, these have been set to one value - discussed below - that best suits our needs. 

## Selected nodes

### Nodes repartition

By considering the physical topology of the room and the disposition of the different nodes, the following repartition was chosen :

![node_rep](experiment-topo.png)

## Interference

By default we define that there will be no interferences during our experiment. 

By adding intra channel interferences, we will decrease the signal noise ratio on the nodes, which  in turn will force them to adopt a multihop communication pattern. Furthermore, it will put the routing protocols in difficulty, amplifying their differences. 

#### Scrambler node

By default the chosen node to generate interference is the node number 5. This node has been chosen in order to maximize the chances for creating a multihop environment.

The second reason it was chosen is because we can only chose Usrp2 or n210 scrambler in order to generate the noise the way we do. 

## Wireless settings

Recall that our goal is to trigger multi-hop communication pattern. Indeed if all communication were on 1 hop, there would be no difference between the two protocols.

In this view, the following settings are recommended:

### Wireless driver

Due to limitations of the intel driver especially regarding establishing an ad hoc network, the current code assumes the use of the `atheros9k` wireless driver.

### Antenna Mask

The default is `1` - meaning, well, to use a single antenna. 

### Physical Rate

In order to limit the communication range (and thus induce multihop communication), the default value of the physical rate is `54 mbps`. This has for effect to limit the signal noise ratio needed in order to have a communication. Indeed the signal for this type of modulation need to be better than with others.

### Transmit Power

Again in order to induce multihop communication, this parameter has been set by default to `5 dBm`. This is because we settled on the Atheros driver, with which this is the minimal applicable value (intel apparently would allow to go down to 0)

### Channel

The default value of this option has been arbitrarily set to `10`. Which is at the end of the 2.4Ghz band.

### Number of ping messages to send

This has been set arbitrarily to 500. Feel free to augment it to generate more statistically correct data's.

## Workflow

### Miscellaneous jupyter notebook magics

In [ ]:
# for convenience, we use this notebook extension
# that will reload any imported python module
# this is handy if you want to use a tex editor to
# change the code in separate python files while you run the notebook
%load_ext autoreload
%autoreload 2

Disabling autoscrolling in cells:

In [ ]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

### Loading various packages required for visualization

In [ ]:
# interactive_output is used to refresh
# a visualization based on user input
from ipywidgets import interactive_output, fixed
from ipywidgets import interactive

In [ ]:
# plotly is one visualization library
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# using plotly in offline mode is a requirement 
# in interactive mode - too slow otherwise
import plotly.offline as pyoff
pyoff.init_notebook_mode()

In [ ]:
from IPython.display import display, HTML

This experiment is composed of 2 stages : 

1. **Data acquisition** : consists in running the code of the experiment on the nodes of R2lab in order to recolt the data. This can be done by launching the `r2labProtocolEval.py` python script or by executing code sections in this notebook. In order to launch the code, you will need at the very least a reservation in the testbed. Please note that running the code is impossible from a publicly hosted notebook.
1. **Data presentation** : consists in visualizing the acquiered data interactively through the notebook.

For convenience, the git repository also contains data from a previous experiment so that you do not have to run the code and you can visualize the results right away.

In [ ]:
#by default we use the already preset data
datadir = 'datasample'
groupdatadir = 'groupdata'
# use the default set of nodes
from constants import DEFAULT_NODE_IDS as on_nodes
print(on_nodes)

*****

## 1. Data acquisition 

The `r2labProtocolEval.py` python script can be used in 3 different ways : 

2. a `one_run` function that run a complete experiment with a specific set of setting.
2. a `all_runs` function, that call `one_run` multiple time in order to do it with all of its given settings.
2. the script itself that calls the `all_runs` function with all the settings givens as command-line options.

All the settings that can be define can be found with the `-h` argument of the pyton script.

In [ ]:
%run -i 'runs' -h

One experiment in the script follows these steps:
* create the folder (given by the -o option, default : logs) needed to store the results.
* verify that your slice name (given by the -s option) has reserved a time slot on the faraday gateway to run an experiment.
* load the image on the nodes inside r2lab (if the -l option is specified) and wait for the nodes to be ready.
* configure the wireless driver and install/activate the routing protocol (specified by the -P option) on each nodes.
* launch tcpdump captures (if and only if the -W is specified).
* wait for the network to settle.
* launch a round of warmup pings (if and only if the -w option is specified) and then settle again.
* get the routing table of each node (if and only if the -M option is specified)
* launch the routing table sampling service (if and only if the -S option is specified)
* launch the data acquisition phase by doing pings from every selected nodes (with the -e option) to every nodes that are `ON` (selected with the -N option).
* kill the routing protocol when the data acquisition phase is finished.
* kill tcpdump and fetch the .pcap on the nodes (if and only if the -W is specified).
* kill the routing table sampling service (if and only if the -S option is specified).
* format the .pcap files generated by tcpdump with tshark on the local machine (if and only if the -W is specified).
* postprocess the results in order to be presented in this notebook. 


The following picture depicts a graph representing the experiment, with some of its optional modules enabled, and a small number of nodes. This png file was obtained by running the bash file named `experiment-graph.run` in the current directory:

![experiment scheduler graph](experiment-graph.png)

In [ ]:
# uncomment if you want to see the command used for generating this graph
!cat experiment-graph.run

### one_run function

This function executes the experiment once, with a given set of environmental conditions that are described as parameters, :

* the routing protocol to use
* the level of interferences to create, if relevant
* the location where to store data (in practical terms, a subdirectory)
* the set of nodes involved:
  * infra nodes (the ones that run the routing deamon)
  * sender nodes
  * receiver nodes
* a precise set of driver/wifi configuration
  * transmision power
  * physical rate
  * antenna mask
  * channel
  
The basic for the scenario, once everything is ready, is to simply run as many pings as there are (sender, receiver) couples defined in the above.

In addition, 5 extra features can be enabled or not, regarding data collection:

* `--warmup` : send various trafic during a warmup period; this is an attempt to help the routing protocol settle in a more effective way
* `--iperf` : run iperf sessions 
* `--map` : **to be confirmed** one-shot collection of routing info on nodes in the experiment
* `--route-sampling` : **to be confirmed** same, but done all along the experiment instead of once
* `--tshark` : runs tcpdump and collect pcap files; warning, this can induce HUGE volumes to be exchanged between the testbed and your laptop.


The condition between  two calls to `one_run` must be:

* no routing protocol running
* no tcpdump running
* no route sampling running
* no interference running

Here we can see some precisions on one_run:

In [ ]:
from runs import one_run

In [ ]:
help(one_run)

At the end, this function fetches all the data into this computer under a directory called `t{}-r{}-a{}-ch{}-I{}-{pro}` containing all the files retrieved from all nodes, i.e., `fitN.pcap`, `PING-E-N` for all N nodes and for all E nodes from which we launch the experiment from.

With:

* `t{}` identifies Tx power for sender nodes in dBm (e.g. 5 to 14):
* `r{}` identifies PHY Tx rate used on all nodes (1 to 54 Mbps)
* `a{}` identifies the antenna mask on all nodes
* `ch{}` denotes the WiFi channel used for transmission
* `I{}` denotes the gain of the noise we generate in the room (None is no noise) in dBm
* `{pro}` identifies the protocol used in this experiment
The postprocessing step consist in invoking the `processmap.py` and `processroute.py` 
to generate intermediate files `rssi-<N>.txt` (one per node, from processmap) and `ROUTES-<N>` (one per selected experimental node, from processroute),
and eventually one consolidated file `RSSI.txt`, that will be used to plot the radiomap.

**IMPORTANT NOTE**: Only Atheros 93xx (with `ath9k` driver) a/b/g/n NICs are supported in these scripts. This is because both cards available on R2lab (namely Atheros and Intel) do no have the same features. In particular, the `iwlwifi` driver for Intel cards limits the number of wireless stations in the same IBSS (parameter `IWLAGN_STATION_COUNT`) to a dozen and it does not support `MESH` mode! Also the Intel 5300 cards are not allowed to use the 5GHz band in Ad Hoc mode.

### all_runs function

This function just executes the `one_run` function with all the combination (cartesian product) of its given parameters.

In [ ]:
from runs import all_runs

In [ ]:
help(all_runs)

For example, instead of expecting a paramater `tx_power` that is a simple string, it expects parameter `tx_powers` that is a list of `tx_power` strings to consider. So for example

    all_runs(tx_powers=[5], phy_rates=[1, 54], antenna_masks=[3, 7])
    
would result in 4 runs of `one_run` with the 2 possible values for `phy_rates` multiplied by the 2 possible values for `antenna_masks`

### Shell script

You can directly run the shell script with python that will simply run all_runs with the given parameters.
As an exemple, you can run this command that will run a dry run for each combinaison of your parameters :

In [ ]:
# uncomment following line to run in dry-run mode
#!./runs.py --dry-run --slice inria_batman --protocol batman -o my-runs -S 1 -D 3 22 37 --warmup --map 

If you remove the `--dry-run` option from that command, and provided that you have a valid slice reservation with username `inria_batman`, then the experiment will run instead of printing its dry-run. Beware that this can take a while...

In [ ]:
customize_table_size_css = """
div.text_cell div.text_cell_render.rendered_html>table td,
div.cell.text_cell.rendered div.text_cell_render.rendered_html>table th
{
    font-size: 15px;
}
"""
HTML(f"<style>{customize_table_size_css}</style>")

You can find here a table illustrating the time take with some combinations of settings. **Note: this table was obtained with an older version of this experiment, and needs to be updated** : 

| Module | Nb Source(s) | Nb Destination(s) | Nb Node On | Time taken |
| :---: | :---: | :---: | :---: | :---: |
| None | 1 | 1 | 10 | 77s |
| TCPDUMP | 1 | 1 | 10 | 90s |
| MAP | 1 | 1 | 10 | 76s |
| Sampling | 1 | 1 | 10 | 79s |
| Warm up | 1 | 1 | 10 | 124s |
| Interference | 1 | 1 | 10 | 82s |
| None | 2 | 10 | 10 | 105s |
| None | 3 | 10 | 10 | 115s |
| All and no load | 3 | 10 | 10 | 347s |
| Load | 1 | 1 | 10 | 281s |

### Run your own

Both `one_run` and `all_runs` function take a `run_name` parameter. That is the name of the directory in which the results are stored. By default the directory used is `datasample`. If you want to run your own experiment end generate your own data, you can modify the following cells:

In [ ]:
# set this to True if you want to run your own data collection experiment

collect_my_data = False

In [ ]:
# if that is the case you need to set these as well

if collect_my_data:
    # your slice name - you'll need a valid reservation
    slicename = "inria_batman"
    # where to store your own data
    datadir = "mydata"
    # the nodes to use
    on_nodes = [1, 4, 12, 15, 19, 27, 31, 33, 37]
    # make sure we load images, at least the first time
    load_images = True

In [ ]:
# run the data collection
if collect_my_data:
    all_runs(
        run_name=datadir,
        slicename=slicename,
        load_images=load_images,
        interferences=None,
        protocols=["batman"],
        node_ids=on_nodes,
        src_ids=on_nodes,
        dest_ids=[37],
        tx_powers=[1], phy_rates=[54],
        antenna_masks=[1], channels=[10],
        dry_run=True,
        )

*****

## Data visualization

This part of the notebook will contain the different result of the experiments. To compare the two protocols, we will interest ourselves in those data :
* RTT
* PDR
* Routing map

### Naming scheme and data organisation

This notebook comes with a pre-populated dataset collected by us, in the `datasample` directory. This directory  contains the data collected during one run of experiments. Two files are important for the visualization:
* PING-X-Y: these files contains the result of the command ping between node X and Y.  They will be used to get the data about RTT and PDR
* ROUTES-X : these files contains the routes present in our network right before sending the pings. They will be used to generate a map of the different routes we have in our network.

These two kinds of files exist for every experiment situated in the `datasample` directory.

In the `datasample` directory, you can find experiment directories. In which you will find the data files.

An experiment directory is formatted like this:

* datasample/t1-r54-a1-ch10-INone-batman/

Where

* `t1` means an emission power of 1dBm

* `r54` means PHY rate=54Mbps

* `a1` means 1 single valid antenna 

* `ch10` means channel 1, i.e. 2457 MHz frequency

* `INone` means no interferences.
* `batman` means that the experiment was done using the B.A.T.M.A.N. protocol

This naming scheme is implemented in a helper function in `r2labProtocolEval`:

In [ ]:
from datastore import naming_scheme
# here's what the naming scheme looks like
naming_scheme(run_name='example-datadir', 
              interference = None, protocol = 'batman')

### Preparation of utilities to parse/format the data

To avoid putting to much non-experiment code inside the notebook, we load some utilities function that will help presenting the data to the plot generating functions.

In [ ]:
# a dashboard allows for interactive manipulation

from dashboards import dashboard_sender, dashboard_receiver

In [ ]:
# if you want to analyze data stored locally in a specific location
# xxx comment out the following line xxx
datadir = "datasample11"

### Visualization of PDR/RTT data

##### Objectives

We use the ` bokeh` library to show the collected data; purpose here is to be show:

* both PDR and RTT data
* for both OLSR and batman

with interactive widgets allowing to tweak the interference level.

##### Principle

The experiment creates various files - mainly the ping outputs stored in files like `PING-01-37`; we scan those files and build for each protocol a pandas dataframe that looks like this:

| index | X | Y | RTT | PDR |
|---------:|---|---|-----|-----|
| 1       | 0 | 5 | 5.3 | 0.01|


and so on with one line per node; the X and Y columns are provided by a r2lab library, and depict the position of nodes on the grid. It is the purpose of `details_from_all_senders` to fill these dataframes (one per protocol).

In [ ]:
from r2labbokeh import init_dataframe_columns, fill_dataframe_from_dict
from datastore import details_from_all_senders

##### The gory details

In [ ]:
from bokeh.plotting import figure as FIGURE, show as SHOW
from bokeh.layouts import row as ROW, column as COLUMN
from bokeh.io import output_notebook, push_notebook

from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool

output_notebook()

In [ ]:
# color palettes used for both data

from bokeh.palettes import Magma256, Inferno

rtt_colors = list(reversed(Inferno[256])) + ["#888888"]
rtt_colormapper = LinearColorMapper(palette=rtt_colors, low=0, high=10)

pdr_colors = Magma256 + ["#888888"]
pdr_colormapper = LinearColorMapper(palette=pdr_colors, low=0., high=1.)

In [ ]:
# how to create one of the 4 figures
def create_figure(datasource, column, color_column, *, colormapper, title, hover, toolbar='left'):
 
    tools = [hover]
    figure = FIGURE(
        title = title,
        plot_width = 450, plot_height=250,
        tools = tools, toolbar_location = toolbar)
    # create the rectangles that make the heatmap
    figure.rect(x='x', y='y', 
                width=1, height=1,
                fill_color={'field': color_column},
                source = datasource)
    return figure        

In [ ]:
#### initialization 

# we can't seem to use infinity, let's use something very big instead
UNAVAILABLE = 10**10
GRAY = '#e0e0e0'
extra_columns = {'PDR' : UNAVAILABLE,
                 'RTT' : UNAVAILABLE,
                 'PDRC' : GRAY,
                 'RTTC' : GRAY,   
                }

def init_bokeh():
    
    # we want to see both data on all views
    hover_tool = HoverTool(
        tooltips=[ ('PDR', '@PDR'), ('RTT', '@RTT ms'), ('node', '@index'), ]
    )
    
    # create 4 figures and arrange them in a 2 x 2 grid
    dfo, dfb = init_dataframe_columns(extra_columns), init_dataframe_columns(extra_columns)
    cdso, cdsb = ColumnDataSource(dfo), ColumnDataSource(dfb)

    fig_pdr_o = create_figure(cdso, 'PDR', 'PDRC', hover=hover_tool,
                              title='PDR to node 37 using OLSR', 
                              colormapper=pdr_colormapper, toolbar='left')
    fig_pdr_b = create_figure(cdsb, 'PDR', 'PDRC', hover=hover_tool,
                              title='PDR to node 37 using batman',
                              colormapper=pdr_colormapper, toolbar='right')
    
    fig_rtt_o = create_figure(cdso, 'RTT', 'RTTC', hover=hover_tool,
                             title='RTT to node 37 using OLSR', 
                             colormapper=rtt_colormapper, toolbar='left')
    fig_rtt_b = create_figure(cdsb, 'RTT', 'RTTC', hover=hover_tool,
                             title='RTT to node 37 using batman',
                             colormapper=rtt_colormapper, toolbar='right')
    
    # show the figure, return handle for updates
    handle = SHOW(COLUMN(ROW(fig_pdr_o, fig_pdr_b),
                         ROW(fig_rtt_o, fig_rtt_b)),
                  notebook_handle=True)

    return (dfo, dfb), (cdso, cdsb), handle

In [ ]:
# create data and figure
dataframes, datasources, handle = init_bokeh()

def update_bokeh(datadir, interference, receiver):
    # the bokeh context is global
    global dataframes, datasources, handle

    dfo, dfb = dataframes
    cdso, cdsb = datasources
    
    details_from_all_senders(dfo, datadir, protocol='olsr', 
                             interference=interference,
                             destination_id=receiver,
                             sources=on_nodes)
    cdso.data = cdso.from_df(dfo)

    details_from_all_senders(dfb, datadir, protocol='batman', 
                             interference=interference,
                             destination_id=receiver,
                             sources=on_nodes)
    cdsb.data = cdsb.from_df(dfb)

    push_notebook(handle)

    
# interactively update it with an UI
dashboard = dashboard_receiver(datadir, continuous_sender=True)
interactive_output(update_bokeh, dashboard)    

###  Routing Map visualization

In [ ]:
from datastore import routing_graph

def routing_graphs(datadir, interference, source):

    dot_batman = routing_graph(
        datadir, interference=interference, source=source, protocol='batman')

    sep = HTML("<hr/>")

    dot_olsr = routing_graph(
        datadir, interference=interference, source=source, protocol='olsr')

    display(dot_batman, sep, dot_olsr)
    
dashboard = dashboard_sender(datadir, continuous_sender=True)
interactive_output(routing_graphs, dashboard)

In [ ]:
# debuggin cell : a glimpse at the olsr dataframe
dfo, dfb = dataframes
# dfo